In [25]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score
from sklearn.pipeline import Pipeline

In [26]:
data = pd.read_csv(r"C:\Users\Aidan\Spotify-Exploration\data\tracks.csv")

In [27]:
data.head(5)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [28]:
data.isnull().sum()

id                   0
name                71
popularity           0
duration_ms          0
explicit             0
artists              0
id_artists           0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
dtype: int64

In [29]:
data[data['popularity'] == 0].value_counts().sum()

44631

In [30]:
data = data[data['popularity'] > 0]

In [31]:
data[data['popularity'] == 0].value_counts().sum()

0

In [32]:
data[data['duration_ms'] == 0].value_counts().sum()

0

In [33]:
data.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,541982.000000,5.419820e+05,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000,541982.000000
mean,29.843386,2.303037e+05,0.046647,0.566804,0.557350,5.225443,-9.957300,0.655880,0.101282,0.423207,0.094577,0.213852,0.554954,118.996470,3.881042
std,17.247144,1.179253e+05,0.210882,0.164606,0.246495,3.521705,4.918093,0.475081,0.173532,0.336427,0.244916,0.185720,0.256752,29.545297,0.455705
min,1.000000,1.470800e+04,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,1.768530e+05,0.000000,0.458000,0.371000,2.000000,-12.553000,0.000000,0.033600,0.087900,0.000000,0.097400,0.349000,96.047000,4.000000
50%,29.000000,2.178270e+05,0.000000,0.580000,0.567000,5.000000,-9.036000,1.000000,0.043600,0.383000,0.000016,0.138000,0.566000,117.976000,4.000000
75%,42.000000,2.651728e+05,0.000000,0.687000,0.758000,8.000000,-6.377000,1.000000,0.074300,0.738000,0.004420,0.278000,0.772000,136.972750,4.000000
max,100.000000,5.621218e+06,1.000000,0.991000,1.000000,11.000000,5.376000,1.000000,0.971000,0.996000,1.000000,1.000000,1.000000,246.381000,5.000000


In [34]:
data['year'] = pd.DatetimeIndex(data['release_date']).year

In [35]:
nums = [i for i in data.columns if data[i].dtype == int or data[i].dtype == float]
nums 

['danceability',
 'energy',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']

In [36]:
import numpy as np 
X = data[nums].values
y = data['popularity'].values.ravel()

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [38]:
lr_pipe = Pipeline([('scaler', StandardScaler()), ('Linear Regression', DecisionTreeRegressor())])

In [39]:
cross_val_score(lr_pipe, X, y, cv=2)

array([-0.3197477 , -0.89831926])

In [40]:
lr_pipe.fit(X_train, y_train)
y_pred = lr_pipe.predict(X_test)
lr_pipe.score(X_test, y_pred)

1.0

In [41]:
print(f"{mean_squared_error(y_test, y_pred)}\n{mean_absolute_error(y_test, y_pred)}")

425.0379820393489
16.131612501897916


In [42]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [43]:
with mlflow.start_run():
        lr = RandomForestRegressor()
        lr.fit(X_train, y_train)

        predicted_qualities = lr.predict(X_test)

        (rmse, mae, r2) = eval_metrics(y_test, predicted_qualities)

        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

  RMSE: 14.61447977462199
  MAE: 11.686834238777353
  R2: 0.27976014407674277
